In [1]:
# Text generation with LSTM
# sampling strategy: reweighting a probability distribution to a different 'temperature'
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):
    '''
    Reweight a probability distribution to increase or decrease entropy.
    # Arguments
        original_distribution: A 1D Numpy array of probability values.
            Must sum to one.
        temperature: Factor quantifying the entropy of the output distribution.
            Higher temperature results in sampling distributions of higher entropy.
        
    # Retures
        A re-weighted version of the original distribution.
    '''
    distribution=np.log(original_distribution)/temperature
    distribution=np.exp(distribution)
    # the sum of the distribution may no longer be 1
    # thus we divide it by its sum to obtain the new distribution
    return distribution/np.sum(distribution)

In [2]:
# implement character-level LSTM text generation
# prepare the data
import keras
import numpy as np

path=keras.utils.get_file('nietzsche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text=open(path).read().lower()
print('Corpus length:',len(text))

# vectorize sequences of characters
maxlen=60
step=3    # sample a new sequence every 'step' characters
sentences=[]
next_chars=[]

for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print('Number of sequences:',len(sentences))

# list of unique characters in the corpus
chars=sorted(list(set(text)))
print('Unique characters:',len(chars))
# dictionary mapping unique characters to their index in 'chars'
char_indices=dict((char, chars.index(char)) for char in chars)

# one-hot encode the characters into binary arrays
print('Vectorization...')
x=np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y=np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i,t,char_indices[char]]=1
    y[i,char_indices[next_chars[i]]]=1

D:\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


606208/600901 [==============================] - 2s 3us/step
Corpus length: 600893
Number of sequences: 200278
Unique characters: 58
Vectorization...


In [3]:
# build the network
from keras import layers

model=keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars),activation='softmax'))
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.01),loss='categorical_crossentropy')

In [4]:
# function for sampling the next character given the model's predictions
def sample(preds, temperature=1.0):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/temperature
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probas=np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
# the text generation loop
import random
import sys

for epoch in range(1, 10):
    print('epoch',epoch)
    # fit the model for 1 epoch on the available training data
    model.fit(x,y,batch_size=128,epochs=1)
    
    # select a text seed at random
    start_index=random.randint(0,len(text)-maxlen-1)
    generated_text=text[start_index:start_index+maxlen]
    print('--- Generating with seed: "'+generated_text+'"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:',temperature)
        sys.stdout.write(generated_text)
        
        # generate 400 characters
        for i in range(400):
            sampled=np.zeros((1, maxlen, len(chars)))
            for t ,char in enumerate(generated_text):
                sampled[0, t, char_indices[char]]=1.
            preds=model.predict(sampled, verbose=0)[0]
            next_index=sample(preds, temperature)
            next_char=chars[next_index]
            generated_text += next_char
            generated_text=generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200278/200278 [==============================] - 290s 1ms/step - loss: 1.6341
--- Generating with seed: "98

=pleasure and social instinct.=--through his relations w"
----- temperature: 0.2
98

=pleasure and social instinct.=--through his relations withing the sense in the self-still and men in the self the same and seek of the sublement in the such a self and all the same and still the sublement of the same and self-deligion of the self-deligion of the self-condition of the sense the self the suberation of the same and all the self-conditions of the same and and still a personal the self-condition of the sense in the subolism and all the con
----- temperature: 0.5
 self-condition of the sense in the subolism and all the conscience in the decement-dears
the presentions and a mode the gearing in its some the devented and
science is the sense it is a putple, the many and a person in the strength and and and best so in lite, and respection of the scholentery of it as a p

being, this agasment.--atparhatiof." woman-severed renly possibine it.--however functous event may been they no a inte" emile speciessowed
one--as allow incoadwate
are joy the conscong peoitatist
extensaucudment, at an and
nations, the same
guach it, as abame for him thoued
firstuniouskeristawity:--lartry bother to the a trreed westheming of vitw
right them and
rearr 
epoch 5
Epoch 1/1
200278/200278 [==============================] - 311s 2ms/step - loss: 1.4717
--- Generating with seed: "of hearing and consequently
spoke all the louder. "he has as"
----- temperature: 0.2
of hearing and consequently
spoke all the louder. "he has as a man and the the the words of the same the man and also the conscious of the procession, the sought to the desires of the conscious of the word and the self-deception of the conscious of the conscious of the proved, the same and a soul, and and the sought to the conscious of the art of the simple, and also the words of the same the states, the same the soug

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


of the most soul, as only as a soul and of the same that the many in the distinguous faith, and the fatherly of a man and philosophy the the sacrifice of far as as he soul as an it of a called period of himself of saint of the same generation of the man has experent of the souls because the distinguous experlition of him the superiority to the another with t
----- temperature: 1.0
ous experlition of him the superiority to the another with the distrinence, which, phanous and sost"--as the injureg? as whithe causifulfulous ortacy
suffering oppars he commandable that phelosely sense of former seveet
didgeats about effact a sourials.ing attempters, as as caustes of an extent 
wan hard
purituous say,
by its here, heals of all that charit, the heavoribility, something just but, how parimsfored, and immediately divited seily, are provined,
----- temperature: 1.2
ow parimsfored, and immediately divited seily, are provined, something every bettay, me them were in him, morality. or a modersg, as

In [10]:
# Deep dream
# load the pre-trained InceptionV3 model
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)
model=inception_v3.InceptionV3(weights='imagenet',include_top=False)

87916544/87910968 [==============================] - 34s 0us/step


In [11]:
# Set up the Dream configuration
model.summary()
layer_contributions={
    'mixed2':0.2,
    'mixed3':3.,
    'mixed4':2.,
    'mixed5':1.5,
}

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, None, None, 3 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, None, None, 3 0           batch_normalization_189[0][0]    
__________________________________________________________________________________________________
conv2d_190

In [13]:
# define the loss to be maximized
layer_dict=dict([(layer.name, layer) for layer in model.layers])

loss=K.variable(0.)
for layer_name in layer_contributions:
    # add the L2 norm of the features of a layer to the loss
    coeff=layer_contributions[layer_name]
    activation=layer_dict[layer_name].output
    
    # avoid border artfacts by only involving non-border pixels in the loss
    scaling=K.prod(K.cast(K.shape(activation),'float32'))
    loss += coeff*K.sum(K.square(activation[:,2:-2,2:-2,:]))/scaling
    
dream=model.input
grads=K.gradients(loss,dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# set up function to retrieve the value of the loss and gradients given an input image
outputs=[loss,grads]
fetch_loss_and_grads=K.function([dream],outputs)

def eval_loss_and_grads(x):
    outs=fetch_loss_and_grads([x])
    loss_value=outs[0]
    grad_values=outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values=eval_loss_and_grads(x)
        if max_loss is not None and loss_value>max_loss:
            break
        print('...Loss value at',i,':',loss_value)
        x += step*grad_values
    return x

In [14]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img=np.copy(img)
    factors=(1, float(size[0])/img.shape[1], float(size[1])/img.shape[2], 1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img=deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)
    
def preprocess_image(image_path):
    img=image.load_img(image_path)
    img=image.img_to_array(img)
    img=np.expand_dims(img, axis=0)
    img=inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format()=='channels_first':
        x=x.reshape((3, x.shape[2], x.shape[3]))
        x=x.transpose((1, 2, 0))
    else:
        x=x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [17]:
import numpy as np

step=0.01      # Gradient ascent step size
num_octave=3   # Number of scales at which to run gradient ascent
octave_scale=1.4  # Size ratio between scales
iterations=20      # Number of steps per scale

max_loss=10.

base_image_path='/Users/think/Downloads/creative_commons_elephant.jpg'
img=preprocess_image(base_image_path)

original_shape=img.shape[1:3]
successive_shapes=[original_shape]
for i in range(1, num_octave):
    shape=tuple([int(dim/(octave_scale**i)) for dim in original_shape])
    successive_shapes.append(shape)
    
# reverse list of shapes, so that they are in increasing order
successive_shapes=successive_shapes[::-1]

# resize the Numpy array to our smallest scale
original_img=np.copy(img)
shrunk_original_img=resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape',shape)
    img=resize_img(img, shape)
    img=gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
    upscaled_shrunk_original_img=resize_img(shrunk_original_img, shape)
    same_size_original=resize_img(original_img, shape)
    lost_detail=same_size_original-upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img=resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_'+str(shape)+'.png')
    
save_img(img, fname='final_dream.png')

Processing image shape (255, 383)
...Loss value at 0 : 1.7892327
...Loss value at 1 : 2.3307054
...Loss value at 2 : 3.0866718
...Loss value at 3 : 3.8079987
...Loss value at 4 : 4.525241
...Loss value at 5 : 5.2030315
...Loss value at 6 : 5.832796
...Loss value at 7 : 6.4598684
...Loss value at 8 : 7.091593
...Loss value at 9 : 7.613576
...Loss value at 10 : 8.1455765
...Loss value at 11 : 8.716384
...Loss value at 12 : 9.212679
...Loss value at 13 : 9.679715


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  # This is added back by InteractiveShellApp.init_path()


Processing image shape (357, 537)
...Loss value at 0 : 2.9886038
...Loss value at 1 : 4.307748
...Loss value at 2 : 5.4929094
...Loss value at 3 : 6.433222
...Loss value at 4 : 7.2584724
...Loss value at 5 : 8.04483
...Loss value at 6 : 8.744614
...Loss value at 7 : 9.4261875
Processing image shape (500, 752)


D:\anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


...Loss value at 0 : 2.854626
...Loss value at 1 : 4.118005
...Loss value at 2 : 5.2332892
...Loss value at 3 : 6.186092
...Loss value at 4 : 7.110281
...Loss value at 5 : 7.9443383
...Loss value at 6 : 8.736466
...Loss value at 7 : 9.492383
